In [97]:
from astropy.coordinates import match_coordinates_sky
from astropy.coordinates import SkyCoord
from astropy.io import ascii
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from math import *
from math import log10
import sep 

In [98]:
def make_filelist(path):
    import os

    file=[]
    for root, directories, files in os.walk(path, topdown=False):  
        for name in files:
            extens = os.path.splitext(name)[-1]
            if extens =='.fits':
                file.append(os.path.join(name))
        
    return file

In [99]:
def make_filelist_locaadd(path):
    import os

    file=[]
    for root, directories, files in os.walk(path, topdown=False):  
        for name in files:
            extens = os.path.splitext(name)[-1]
            if extens =='.fits':
                file.append(os.path.join(root, name))
        
    return file

In [100]:
def match_radec(filename, catalname):
    
    catal=ascii.read(catalname)
    ra = catal['raMean']
    dec = catal['decMean']
    inflag = catal['objInfoFlag']
    qflag = catal['qualityFlag']
    c = SkyCoord(ra, dec, frame='icrs', unit='deg')

    ob = ascii.read(filename)    
    ALPHA_J2000 = ob['ALPHA_J2000']
    DELTA_J2000 = ob['DELTA_J2000']
    flag = ob['FLAGS']
    o = SkyCoord(ALPHA_J2000, DELTA_J2000, frame='icrs', unit='deg')

    indx, sep, _ = o.match_to_catalog_sky(c)
    return sep, c, o, flag, inflag, qflag, ob, catal, indx

In [101]:
def flagging(sep_limit, sep, c, o, flag, inflag, qflag):
    #seperation and flag restriction 
    P=[]
    #sep_limit=2

    for i in range(len(o)):
        if sep.arcsec[i]<sep_limit:
            if flag[i]<1:
                if inflag[i] < 10**9:
                    if qflag[i] < 100:
                        P.append(i)
        
    #print(P)
    return P

In [102]:
def zeropoint(P, ob, catal, indx):
    mag_ob = np.array(ob['MAG_APER'])
    mag_catal = np.array(catal['R'])
    magErr = np.array(ob['MAGERR_APER'])
    gErr = np.array(catal['gMeanKronMagErr'])
    rErr = np.array(catal['rMeanKronMagErr'])
    FWHM = np.array(ob['FWHM_IMAGE'])

    #zero point & zero point error calculation 
    zp = []
    zpE = []
    for i in range(len(mag_ob)):
        u=indx[i]
        x = float(mag_catal[u])-float(mag_ob[i])
        e = np.sqrt(magErr[i]**2 + gErr[u]**2 + rErr[u]**2)
        zp.append(x)
        zpE.append(e)
    
    #print(zp)
    #zp versus ref mag
    mag_c_new=[]
    zp_new=[]
    zpE_new=[]
    seeing=[]
    for i in P:
        u=indx[i]
        if 28>zp[i]>24:
            if zpE[i]<0.1:
                if mag_catal[u]>13:
                    if FWHM[i]*0.4454<5:
                        mag_c_new.append(mag_catal[u])
                        zpE_new.append(zpE[i])
                        zp_new.append(zp[i])
                        seeing.append(0.4454*FWHM[i])
    
    return zp_new, zpE_new, mag_c_new, seeing

In [162]:
def depth(ob, zp_new, seeing, image_file):
    import sep
    
    Me = ob['MAGERR_APER']
    M = ob['MAG_APER']
    

    #image_data=fits.open('/data3/yunyi/SOAO/fitsfiles/M101_20210604/Calib-SOAO-M101-20210604-153827-R-60.fits')
    #header=image_data[0].header
    #zp= header['ZP_0']
    #print(zp_new)
    zp=np.mean(zp_new)

    
    data = fits.getdata(image_file,ext=0)
    data = data.byteswap().newbyteorder()
    bkg=sep.Background(data, bw=512, bh=512)
    
    N=5
    std = bkg.globalrms
    slv = bkg.globalback
    r=np.mean(seeing)
    dp=zp - 2.5*log10(N*std*np.sqrt(np.pi*(r**2)))

    print('analytical depth is %f'%dp)
    print('seeing is %f'%r)
    print('std is %f'%std)

    return std, dp, r, slv

In [232]:
def headeradd(catloca):
    import os
    
    os.chdir('/data3/yunyi/SOAO/select/NGC0514')
    filelist = make_filelist('/data3/yunyi/SOAO/select/NGC0514')
    filelist_locaadd = make_filelist_locaadd('/data3/yunyi/SOAO/select/NGC0514')
    avg_seeing=[]
    avg_depth=[]
    avg_std=[]
    for i in range(len(filelist)):
        os.chdir('/data3/yunyi/SOAO/select/NGC0514')
        os.system(f'cp {filelist[i]} /data3/yunyi/SOAO')
        os.chdir('/data3/yunyi/SOAO')
        seeingval = 1.8
        apt_pix = seeingval*2/0.4454
        os.system(f'sex {filelist[i]} -PHOT_APERTURES {apt_pix}')
        sep, c, o, flag, inflag, qflag, ob, catal, indx=match_radec('/data3/yunyi/SOAO/test.cat', catloca)
        sep_limit=1
        P=flagging(sep_limit, sep, c, o, flag, inflag, qflag)
        zp_new, zpE_new, mag_c_new, seeing =zeropoint(P, ob, catal, indx)
        
        std, dp, r, slv = depth(ob, zp_new, seeing, filelist_locaadd[i])
        std = float(std)
        dp = float(dp)
        r = float(r)
        slv=float(slv)
        
        if np.isnan(r) != True:
            fits_file = filelist_locaadd[i]
            print(fits_file)
            print(r)
            print(dp)
            print(std)
            avg_seeing.append(r)
            avg_depth.append(dp)
            avg_std.append(std)
            '''
            with fits.open(fits_file, 'update') as f:
                for hdu in f:
                    hdu.header['SEEING'] = r
                    hdu.header['UL5_2'] = dp
                    hdu.header['SKYSIG'] = std
                    hdu.header['SKYVAL'] = slv
                    print(hdu.header)

        
    os.chdir('/data3/yunyi/SOAO')
    os.system('rm Calib*')
    '''
    print('mean seeing is %f'%np.mean(avg_seeing))
    print('mean depth is %f'%np.mean(avg_depth))
    print('mean std is %f'%np.mean(avg_std))
    


In [233]:
headeradd('/data3/yunyi/SOAO/code/NGC0514_R.csv')

analytical depth is 19.573395
seeing is 1.687557
std is 17.046768
/data3/yunyi/SOAO/select/NGC0514/Calib-SOAO-NGC0514-20191010-173505-R-60.fits
1.6875569714285714
19.573394721245986
17.046768188476562
analytical depth is 19.409759
seeing is 1.897521
std is 17.071972
/data3/yunyi/SOAO/select/NGC0514/Calib-SOAO-NGC0514-20191010-173610-R-60.fits
1.8975212105263157
19.409758738005088
17.071971893310547
analytical depth is 19.389861
seeing is 1.917578
std is 17.032520
/data3/yunyi/SOAO/select/NGC0514/Calib-SOAO-NGC0514-20191010-173715-R-60.fits
1.9175780000000002
19.389860852652443
17.032520294189453
analytical depth is 19.438369
seeing is 1.891265
std is 16.998016
/data3/yunyi/SOAO/select/NGC0514/Calib-SOAO-NGC0514-20191010-173820-R-60.fits
1.8912647027027027
19.438368972406785
16.998016357421875
analytical depth is 19.388198
seeing is 1.989453
std is 15.307314
/data3/yunyi/SOAO/select/NGC0514/Calib-SOAO-NGC0514-20201111-110553-R-60.fits
1.9894533333333333
19.38819814463897
15.307313919067